# 01 - Relevant Python 3.10 Changes

## Structural Pattern Matching

The release of Python 3.10 has brought some new features.

This is a summary of the ones _I_ deemed relevant to this course, and does **not** include all the changes!

For full release details, see [here](https://docs.python.org/3/whatsnew/3.10.html)

One thing I often hear people ask, is, what's the Python equivalent of a `switch` statement. Until now, the answer has alwasy been - there isn't one. Use `if...elif` constructs. Python 3.10 introduces a new language element (`match`) to implement something called **pattern matching**, that can be used to replicate this `switch` behavior you might be used to in other languages.

In [21]:
def respond(language):
    match language:
        case "Java" | "Javascript":                # We can add OR statements into the case
            return "Hmm, coffee!"
        case "Python":
            return "I'm not scared of snakes!"
        case "Rust":
            return "Don't drink too much water!"
        case "Go":
            return "Collect $200"
        case _:                                    # This is the default return if no matches are found. This catchall is called a "wildcard"
            return "I'm sorry..."

In [22]:
respond('Python') # we are case-sensitive

"I'm not scared of snakes!"

In [23]:
respond('someotherlanguage')

"I'm sorry..."

In [24]:
respond('Java')

'Hmm, coffee!'

This next example won't be as easy to understand because it contains topics from parts other than this part in the deep dive series, but I'll annotate it as much as I can:

Suppose we have some kind of command language for driving a remote controlled robot in a maze, picking up and dropping items as it moves around. Our robot is very simple, it can move in only a few directions, and one step at a time. So to move forward three spaces, we would issue three `move forward` commands.

Additional commands are `move backward`, `move left`, `move right`. We also have a few other commands our robot understands: `pick` and `drop` for picking up and dropping objects it might find.

Let's start by using some symbols to represent the robot's actions:

In [25]:
symbols = {
    "F": "\u2192", 
    "B": "\u2190", 
    "L": "\u2191", 
    "R": "\u2193", 
    "pick": "\u2923", 
    "drop": "\u2925"
}

symbols

{'F': '→', 'B': '←', 'L': '↑', 'R': '↓', 'pick': '⤣', 'drop': '⤥'}

Here is a less-than-elegant way:

In [26]:
def op(command):
    match command:
        case "move F":
            return symbols["F"]
        case "move B":
            return symbols["B"]
        case "move L":
            return symbols["L"]
        case "move R":
            return symbols["R"]
        case "pick":
            return symbols["pick"]
        case "drop":
            return symbols["drop"]
        case _:
            raise ValueError(f"{command} does not compute!")
            
[op("move F"), op("pick"), op("move R"), op("drop")]

['→', '⤣', '↓', '⤥']

We could use something called **capturing** matched sub-patterns to simply our code somewhat. Here's a slightly more elegant way:

In [27]:
def op(command):
    match command:
        case ["move", ("F" | "B" | "L" |"R") as direction]: # We pass in a list where the first element is "move" and the 2nd is either F, B, L or R. 
                                                            # If it's one of these, then this value is stored in the 'direction' variable.                                                 
            return symbols[direction]
        case "pick":
            return symbols["pick"]
        case "drop":
            return symvols["drop"]
        case _:
            raise ValueError(f"{command} does not compute!")
            
[op(["move", "L"]), op(["move", "R"]), op(["move", "B"]), op("pick"), op(["move", "L"])]

['↑', '↓', '←', '⤣', '↑']

The lack of consistency with using lists for state action + direction but not for just an action such as pick makes this less elegant. Also, we would like to have something that looks more like: op(move up up down left), op(pick), op(move left right down). Here's an even more elegant way: 

In [28]:
def op(command):
    match command:
        case ['move', *directions]:                                       # This allows us to pass multiple comma-separated arguments.
            return tuple(symbols[direction] for direction in directions)  # This takes each argument, converts it to a symbol and packs them all into a tuple
        case "pick":
            return symbols["pick"]
        case "drop":
            return symbols["drop"]
        case _:
            raise ValueError(f"{command} does not compute!")

In [29]:
[op(["move", "F", "F", "L"]), op("pick"), op(["move", "R", "L", "F"]), op("drop")]

[('→', '→', '↑'), '⤣', ('↓', '↑', '→'), '⤥']

One issue is that if we pass an illegal direction, we would get a generic error - not the value error we see in the wildcard case. We would rather just get our custom `ValueError`. To do this we can place a **guard** on our `case` for the `move` command, that will not only do the match but also test an additional condition. This is the most elegant way:

In [30]:
def op(command):
    match command:
        case ['move', *directions] if set(directions) < symbols.keys(): # 'set' removes repeats in our indefinite list of directions. Then, we check if our set 
                                                                        # of directions is a subset of the original set of directions.
            return tuple(symbols[direction] for direction in directions) # We only reach this line if the condition is met.
        case "pick":
            return symbols["pick"]
        case "drop":
            return symbols["drop"]
        case _:
            raise ValueError(f"{command} does not compute!")
            
[op(["move", "F", "F", "L"]), op("pick"), op(["move", "R", "L", "F"]), op("drop")]

[('→', '→', '↑'), '⤣', ('↓', '↑', '→'), '⤥']

That `if ` statement (the **guard**) will only let the case block execute if the match is true **and** that `if` expression evaludates to `True`:

## The `zip` Function

We may recall that a `zip` will stop when one of the iterables are exhausted. Note that `zip` is a generator function so we enclose it in `list` to generate the terms.

In [31]:
l1 = ['a', 'b', 'c']
l2 = [10, 20, 30, 40]

list(zip(l1, l2))

[('a', 10), ('b', 20), ('c', 30)]

In `itertools` from the standard library we have `zip_longest` which allows us to continue until the last iterable is exhausted and just fill in a default value for the shorter iterables' terms.

In [32]:
from itertools import zip_longest

list(zip_longest(l1, l2, fillvalue = 'No value found'))

[('a', 10), ('b', 20), ('c', 30), ('No value found', 40)]

But what if we only want to begin the generating if the iterables are the same length? We may add a conditional that checks if they're the same length but the issue is that imposing `len` on an iterable will automatically exhaust the iterable, meaning there will be nothing to zip afterwards. Our regular `zip` has a solution which is implemented by a parameter called `strict`. If set to True, the generating of terms will only execute if the lists are the same length, otherwise return a ValueError. This should be used in pretty much all cases because usually we zip together lists that we expect to be the same length.

In [33]:
list(zip(l1, l2, strict=True))

ValueError: zip() argument 2 is longer than argument 1

# 02 - Relevant Python 3.9 Changes

## Timezones 

We don't cover 3rd party libraries in this course, but if you've worked with Python in a production environment, you will likely have come across the dreaded timezone and Daylight Savings issues that plague datetimes!

Most likely you will have resorted to using the `pytz` and `python-dateutil` libraries to help with that. Now, Python 3.9 is proud to introduce the `zoneinfo` module to deal with timezones properly. About time too!

For full info on this, refer to [PEP 615](https://peps.python.org/pep-0615/).

And the Python [docs](https://docs.python.org/3.9/library/zoneinfo.html#module-zoneinfo).

**Windows Users**: you will likely need to add a dependency on the `tzdata` [library](https://pypi.org/project/tzdata/) for the IANA time zone database. See [this note](https://docs.python.org/3.9/library/zoneinfo.html#data-sources)

You should also take a look at this [presentation](https://pganssle-talks.github.io/chipy-nov-2020-zoneinfo/#/) by Paul Ganssle who wrote that module - very interesting read!

Let's look at how we might have handled timezone and DST using `pytz` and `dateutil`, and contrast that to how we can use the new `zoneinfo` module instead.

In [34]:
import zoneinfo                             # New module
from datetime import datetime, timezone
from zoneinfo import ZoneInfo

import dateutil                             # 3rd party old module; WON'T BE USED IN THE NOTEBOOK
import pytz                                 # 3rd party old module

In [35]:
# OLD WAY

for tz in pytz.all_timezones[:5]: # It's a long list; I'm gonna show the first 5.
    print(tz)

Africa/Abidjan
Africa/Accra
Africa/Addis_Ababa
Africa/Algiers
Africa/Asmara


In [36]:
# NEW WAY

for tz in sorted(zoneinfo.available_timezones())[:5]:
    print(tz)

Africa/Abidjan
Africa/Accra
Africa/Addis_Ababa
Africa/Algiers
Africa/Asmara


Let's say we want to convert from one timezone to another, say UTC (the international time in which all other timezones are represented by their positive/negative offsets from it) to the time in Melbourne, Australia. For some reason, both old and new are not working with mine as well as the original jupyter notebook.

Note: 'naive' time means a time that doesn't have a timezone associated with it.

In [37]:
# OLD WAY

now_utc_naive = datetime.utcnow()
now_utc_aware = now_utc_naive.replace(tzinfo=timezone.utc) # tzinfo is just an optional kwarg that contains the timezone; adding it makes a time 'aware'.

print(f"The current time in UTC is {now_utc_aware}")

now_utc_aware.astimezone(pytz.timezone('Australia/Melbourne')) #astimezone is a method from the datetime object.

print(f"The current time in Melbourne is {now_utc_aware}")

The current time in UTC is 2022-11-08 11:40:48.882948+00:00
The current time in Melbourne is 2022-11-08 11:40:48.882948+00:00


In [38]:
# NEW WAY

print(f"The current time in UTC is {now_utc_aware}")

tz_zoneinfo_dublin = ZoneInfo('Europe/Dublin') # We get info of the timezone from Python's ZoneInfo, not from pytz.

now_utc_aware.astimezone(tz_zoneinfo_dublin)

print(f"The current time in Europe/Dublin is {now_utc_aware}")

The current time in UTC is 2022-11-08 11:40:48.882948+00:00
The current time in Europe/Dublin is 2022-11-08 11:40:48.882948+00:00


## Math

Made greatest common divisor and least common multiple functional for more than 2 arguments.

## Dictionary Unions

Before if we wanted to join two dictionaries together, we'd have to unpack them both into a new dictionary. If there were any repeats of a key, the last mention will be the one that's kept. The intuitive way of joining iterables together is via concatenation, e.g.:

In [39]:
[1, 2, 3] + [4, 5, 6]

[1, 2, 3, 4, 5, 6]

We can now use this concatenation method for dictionaries but using the union symbol, |. 

In [40]:
d1 = {'a':1, 'b':2, 'c':3}
d2 = {'c':300, 'd':400, 'e':500}

# OLD

print("The old way of concatenating dictionaries: {}".format({**d1, **d2})) #Can't use f-strings on this

# NEW

print(f'The new way of concatenating dictionaries: {d1 | d2}')

The old way of concatenating dictionaries: {'a': 1, 'b': 2, 'c': 300, 'd': 400, 'e': 500}
The new way of concatenating dictionaries: {'a': 1, 'b': 2, 'c': 300, 'd': 400, 'e': 500}


## String Methods

If we want to remove a prefix/suffix from each element in a list, we would normally use list comprehension, which is a fine way to do it. Let's say we want to remove '(log) ' from each item.

In [41]:
data = [
    "(log) [2022-03-01T13:30:01] Log record 1",
    "(log) [2022-03-01T13:30:02] Log record 2",
    "(log) [2022-03-01T13:30:03] Log record 3",
    "(log) [2022-03-01T13:30:04] Log record 4",
]

In [42]:
# OLD

[item.replace('(log) ','') for item in data]

['[2022-03-01T13:30:01] Log record 1',
 '[2022-03-01T13:30:02] Log record 2',
 '[2022-03-01T13:30:03] Log record 3',
 '[2022-03-01T13:30:04] Log record 4']

In [43]:
# NEW

[item.removeprefix("(log) ") for item in data]

['[2022-03-01T13:30:01] Log record 1',
 '[2022-03-01T13:30:02] Log record 2',
 '[2022-03-01T13:30:03] Log record 3',
 '[2022-03-01T13:30:04] Log record 4']

# 03 - Python 3.8 - Assignment Expressions

Another enhancement to the Python core language that was introduced in Python 3.8 is **assignment expressions**.

Remember that an **expression** is simply a snippet of code that is evaluated, e.g. `1+2`. We assign it to a variable with an equals.

So what are **expression assignments**?

Expression assignments allows us to assign expressions to a variable **inside** an expression, using the `:=` operator (the so-called *walrus* operator).

To expound, we have two separate things: **expressions** and **assignments**.
- expressions have a return value but no handle to it.
- assignments have no return but a handle to it (via a variable).

Here is an expression:

In [44]:
1 + 2 

3

Here is an assignment:

In [45]:
x = (1+2)

We can combine the two to form an **assignment expression** which evaluates the expression with a return whilst also providing a handle. We need the walrus operator to do it and the brackets.

In [46]:
(x := 1 + 2)

3

In [47]:
x # we have access to x

3

This still behaves like an expression because it is - and the brackets should remind you of that - which means that we can nest these to perform an assignment to this 'expression'

In [48]:
a = (x := 10 + 20)

In [49]:
print(a)

30


So, we evaluate the brackets in the regular way which evaluates to 30 for `x` and returns 30 to `a`.

Let's see how this assignment expression works when we deal with mutable objects such as lists:

In [50]:
l1 = (l2 := [1, 2] + [3, 4])

In [51]:
id(l1) == id(l2)

True

Why's this useful? Often, we end up writing expressions in terms of other sub expressions, not just for clarity, but sometimes to **avoid repeating** function calls or expression evaluations.

Example:

In [52]:
import time
import math

def slow_function(x, y):
    time.sleep(0.5)
    return round(math.sqrt(x**2 + y**2))

In [53]:
from time import perf_counter

start = perf_counter()
even_results = []
for i in range(10):
    if slow_function(i, i) % 2 == 0:
        even_results.append(slow_function(i, i))
end = perf_counter()
print(even_results)
print(f'Elapsed: {end - start:.1f} seconds')

[0, 4, 6, 8, 10]
Elapsed: 7.5 seconds


This is slower because we make a function call within the `if` block and then once more within the `append()`, so we'd naturally create a variable for this to prevent repeating ourselves.

In [54]:
start = perf_counter()
even_results = []
for i in range(10):
    result = slow_function(i, i)
    if result % 2 == 0:
        even_results.append(result)
end = perf_counter()
print(even_results)
print(f'Elapsed: {end - start:.1f} seconds')

[0, 4, 6, 8, 10]
Elapsed: 5.0 seconds


All we're doing in this code is building up a list from an iterable (`range(10)`) and throwing away any elements that are odd. Seems like a good place to use list comprehension. But... we can't write that `result = slow_function(i, i)` in our list comprehension - so we would be back to the original (and slower) may of doing it:

In [55]:
start = perf_counter()

even_results = [slow_function(i, i) for i in range(10) if slow_function(i, i) % 2 == 0] # 2 function calls; not good.

end = perf_counter()
print(even_results)
print(f'Elapsed: {end - start:.1f} seconds')

[0, 4, 6, 8, 10]
Elapsed: 7.5 seconds


And this is where the assignment expression operator comes in very handy:

In [56]:
start = perf_counter()

even_results = [result for i in range(10) if (result := slow_function(i, i)) % 2 == 0]

end = perf_counter()
print(even_results)
print(f'Elapsed: {end - start:.1f} seconds')

[0, 4, 6, 8, 10]
Elapsed: 5.0 seconds


Notice that we can't write the assignment expression in the first part and reference it in 2nd. This is because the conditional `if` is executed first.

In [57]:
del result

even_results = [(result := slow_function(i, i)) for i in range(10) if result % 2 == 0] 

NameError: name 'result' is not defined

There are 2 more examples in the full notes, but this assumes some understanding of generators.

# 04 - Relevant Python 3.8_3.7 Changes

## Force Positional

Recall that if within a function argument we see `*`, this indicates to us that every parameter afterwards MUST be a keyword argument. In other words, we **force a keyword argument**. In the below, `c` has been forced as a keyword argument.

In [58]:
def func(a, b, *, c):
    print(a + b + c)
    
# func(3, 4, 5) # doesn't work; expected 2 positional arguments, got 3.

func(a=3, b=4, c=5) # less common but works

func(3, 4, c=5) # works

12
12


We can now force the use of purely positional arguments only by using `/` instead of `*`. Arguments prior to this must be passed as positonal arguments. If they're passed as keyword, it won't work

In [59]:
def func(a, b, /, c):
    print(a + b + c)

func(3, 4, 5) # works; a and b MUST be positional, c can be positional or keyword
func(3, 4, c=5) # works; a and b MUST be positional, c can be positional or keyword

func(a=3, b=4, c=5) # doesn't work; a and b are not positional.

12
12


TypeError: func() got some positional-only arguments passed as keyword arguments: 'a, b'

We can still use `*`. For example, let's say we want to create two parameter's that must be positional (a and b) and one parameter (c) that must be keyword:

In [60]:
def func(a, b, /, *, c):
    print(a + b + c)
    
func(3, 4, c=5) # no other variations are allowed.

12


## f-string enhancements

If we are using f-strings strictly for saying expressing a variable and its evaluation, we now have a shorthand by replacing `a={a}` with `{a=}`. We can also apply a type conversion like str() or int() by replacing `{a=}` with `{a=:}` - these are known as format specifiers and already exist elsewhere in Python.

In [61]:
# OLD

a, b = 'Hello', 357.72

print(f"a={a}, b={b}")

a=Hello, b=357.72


In [62]:
# NEW

a, b = 'Hello', 3

print(f"{a=:s}, {b=:.2f}") #:.2f applies 2 decimal places conversion

a=Hello, b=3.00


# 05 - Python 3.6 Highlights

Lots of stuff in here are things you're already familiar with like numeric literals (`1_000_000` is interpreted as `1000000`) and f-strings ,but note that we can actually use format specifiers such as 2 decimal place e.g. `f"My value rounded is {result:.2f}"`.

There's also type hints which I think you should get used to doing, but remember there's no static typing here. Example:

In [63]:
def squares(l: list[int]) -> list[int]: # l should be type 'list' and its going to contain integers. It will return (->) a list of integers
    return [i ** 2 for i in l]

my_list: list[int] = [1, 2, 3, 4] # my_list, its a list of integers 

# 06 - Python 3.6 - Dictionary Ordering

Works intuitively; rarely a need for ordered dict now. If you want to know some basic dictionary mutations such as removing the first/last item, getting one key at a time in order from the dictionary etc., read the full notes. I've jotted some down here.

In [64]:
# Pop last item

d = {'a':1, 'b':2, 'c':3, 'x':100, 'y':200}
print('start:', d)
d.popitem()
print('pop last item:', d)

start: {'a': 1, 'b': 2, 'c': 3, 'x': 100, 'y': 200}
pop last item: {'a': 1, 'b': 2, 'c': 3, 'x': 100}


In [65]:
# Pop first item; first, we need to find the first key

d = {'a':1, 'b':2, 'c':3, 'x':100, 'y':200}
print('start:', d)
key = next(iter(d.keys()))
d.pop(key)
print('pop first item:', d)

start: {'a': 1, 'b': 2, 'c': 3, 'x': 100, 'y': 200}
pop first item: {'b': 2, 'c': 3, 'x': 100, 'y': 200}


# 07 - Python 3.6 - Preserved Order of kwargs - Named Tuple Application

Works intuitively; not much to say.

# 08 - Python 3.6 - Underscores and Numeric Literals

Another format specifier I learn't here is `:_` which adds an underscore after every 1000:

In [66]:
value = 10000000000000
f"{value:_}"

'10_000_000_000_000'

# 09 - Python 3.6 - f-Strings

Already familiar with this.